# Step 10.2: Create Model Files

## Objective

In this step of the project we will train the model on the whole dataset and create the pickle files that will be deployed on the API.

In [1]:
import aux_functions
from app_functions import attempt_predict
from transformers import TimeTransformer2, BoolTransformer, lat_lon_imputer, Group_Age_Range, Group_Ethnicity

import json
import joblib
import pickle
import requests
from time import sleep
import random
import sqlalchemy
import psycopg2

import pandas as pd
pd.set_option('display.max_columns', 100)
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import numpy as np
import datetime as dt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import cross_val_score
# from category_encoders import OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder, RobustScaler, OrdinalEncoder
from category_encoders.target_encoder import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB, CategoricalNB

from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler

from sklearn.utils import resample
from sklearn.metrics import precision_score, recall_score, f1_score,  accuracy_score, roc_auc_score, make_scorer, confusion_matrix, roc_curve

# needed to use matplotlib inside jupyter notebook
%matplotlib inline 

# Get the data

In [2]:
df_ = pd.read_csv("data/train.csv")

drop_cols = ['Self-defined ethnicity', 'Outcome', 'Outcome linked to object of search', 'Removal of more than just outer clothing']

df_clean = aux_functions.clean_data(df_, drop_cols)

# Pipeline Definition

In [3]:
def create_pipeline(df, model):

    y = df["target"].copy()
    X = df.drop(columns=["target"]).copy()


    time_cols = ['Date']

    categorical_cols = ['Type', 'Object of search']
    
    bool_cols = ['Part of a policing operation']

    numerical_cols = ['Latitude', 'Longitude', 'station'] # station is needed for NaN imputing

    # Define timeseries pipeline
    time_pipe = Pipeline([
            ('time_transformer', TimeTransformer2()),
            ('scaler', MinMaxScaler())
        ])
    
    
    # Define categorical pipeline
    cat_pipe = Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', TargetEncoder(handle_unknown='value', min_samples_leaf=20 , smoothing=10))
        ])
    
    # Define boolean pipeline
    bool_pipe = Pipeline([
            ('bool_transformer', BoolTransformer())
        ])

    # Define numerical pipeline
    numeric_pipe = Pipeline([
            ('imputer', lat_lon_imputer()),
            ('extra_imputer', SimpleImputer(strategy='mean')),
            ('scaler', MinMaxScaler())
        ])

    
    # Combine categorical and numerical pipelines
    preprocessor = ColumnTransformer([
        ('time_transformer', time_pipe, time_cols),
        ('cat', cat_pipe, categorical_cols),
        ('bool', bool_pipe, bool_cols),
        ('num', numeric_pipe, numerical_cols)],
    remainder='drop')


    # Fit a pipeline with transformers and an estimator to the training data
    pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('model', model)
        ])

    rus = RandomUnderSampler(random_state=42)
    X_resampled, y_resampled = rus.fit_resample(X, y)

    pipeline.fit(X_resampled, np.ravel(y_resampled))
    # pipeline.fit(X, np.ravel(y))


    return pipeline, X_resampled, y_resampled
    # return pipeline, X, y


def see_cross_val(pipeline, X, y):

    prec_scorer = make_scorer(precision_score)

    prec_scores = cross_val_score(pipeline, X, np.ravel(y), cv=5, scoring=prec_scorer)
    print(prec_scores)

# Train the Model with all the Data

In [4]:
# Check transformed data

model = GaussianNB()

pipeline, X, y = create_pipeline(df_clean, model)

see_cross_val(pipeline, X, y)

[0.52007437 0.54811129 0.51345917 0.5432847  0.53321139]


# Extract Model Files

In [5]:
with open("columns.json", 'w') as fh:
    json.dump(X.columns.tolist(), fh)


with open("dtypes.pickle", 'wb') as fh:
    pickle.dump(X.dtypes, fh)

joblib.dump(pipeline, "pipeline.pickle") 

['pipeline.pickle']

# Test the App

In [3]:
with open('data/trial_moment_1.json') as moment_1:
  parsed_json = json.load(moment_1)

print(parsed_json[1]["data"])

for n in range(len(parsed_json)):
  print(parsed_json[n]["data"])

{'observation_id': 'eeb891e3-3913-4590-82a9-dc23c212dceb', 'Type': 'Person search', 'Date': '2022-04-18T22:24:57+00:00', 'Part of a policing operation': nan, 'Latitude': nan, 'Longitude': nan, 'Gender': 'Male', 'Age range': '10-17', 'Officer-defined ethnicity': 'Other', 'Legislation': 'Police and Criminal Evidence Act 1984 (section 1)', 'Object of search': 'Articles for use in criminal damage', 'station': 'city-of-london'}
{'observation_id': '1558a55e-3df2-4665-8beb-0f0c5eaa0408', 'Type': 'Person search', 'Date': '2022-04-06T18:25:00+00:00', 'Part of a policing operation': True, 'Latitude': nan, 'Longitude': nan, 'Gender': 'Male', 'Age range': '10-17', 'Officer-defined ethnicity': 'White', 'Legislation': 'Misuse of Drugs Act 1971 (section 23)', 'Object of search': 'Controlled drugs', 'station': 'nottinghamshire'}
{'observation_id': 'eeb891e3-3913-4590-82a9-dc23c212dceb', 'Type': 'Person search', 'Date': '2022-04-18T22:24:57+00:00', 'Part of a policing operation': nan, 'Latitude': nan, 

In [2]:
APP_NAME = 'ldsacapstone-production.up.railway.app'

In [11]:
# testing the predict endpoing

url = f"https://{APP_NAME}/should_search/"

for n in range(len(parsed_json)):

    payload = parsed_json[n]["data"]

    r = requests.post(url, data=json.dumps(payload), headers={"Content-Type": "application/json"})
    display(r)
    display(r.content)




<Response [200]>

b'{\n  "outcome": "[ True]"\n}\n'

<Response [200]>

b'{\n  "outcome": "[False]"\n}\n'

<Response [200]>

...

b'{\n  "outcome": "[False]"\n}\n'

<Response [200]>

b'{\n  "outcome": "[False]"\n}\n'

<Response [200]>

b'{\n  "outcome": "[ True]"\n}\n'

In [14]:
with open('data/trial_moment_2.json') as moment_1:
  parsed_json = json.load(moment_1)

print(parsed_json[1]["data"])

for n in range(len(parsed_json)):
  print(parsed_json[n]["data"])

{'observation_id': 'eeb891e3-3913-4590-82a9-dc23c212dceb', 'outcome': True}
{'observation_id': '1558a55e-3df2-4665-8beb-0f0c5eaa0408', 'outcome': False}
{'observation_id': 'eeb891e3-3913-4590-82a9-dc23c212dceb', 'outcome': True}
{'observation_id': '898d6606-c55b-4a54-9480-f967beaff1cf', 'outcome': False}
{'observation_id': '3ff08b3c-c1fc-4c9f-97fe-470cf3a61cef', 'outcome': False}
{'observation_id': '73d7c589-7605-42ab-9c5c-d0fbb897adb0', 'outcome': False}
{'observation_id': '1d276941-2dd7-4cc0-b14d-764ee8bcd966', 'outcome': False}
{'observation_id': 'fd0a32a8-1672-45da-bf93-b93524441b24', 'outcome': True}
{'observation_id': '15af9195-3fa0-40aa-9083-da7767f0c801', 'outcome': True}
{'observation_id': '9ac66922-d13b-4877-8bab-41ef925d3bb8', 'outcome': True}
{'observation_id': '27dc412f-64e2-4245-9a25-db0514b2e980', 'outcome': True}
{'observation_id': 'aeed813b-d4e9-489d-81b3-9b8ec8f62d52', 'outcome': False}
{'observation_id': 'caabe84e-97e0-4305-b540-eda6d56e51f3', 'outcome': True}
{'obse

In [17]:
# Testing the /update endpoint

url = f"https://{APP_NAME}/search_result/"
for n in range(len(parsed_json)):

    payload = parsed_json[n]['data']

    r = requests.post(url, data=json.dumps(payload), headers={"Content-Type": "application/json"})
    display(r)
    display(r.content)


<Response [200]>

b'{\n  "observation_id": "1558a55e-3df2-4665-8beb-0f0c5eaa0408", \n  "outcome": "False", \n  "predicted_outcome": "True"\n}\n'

<Response [200]>

b'{\n  "observation_id": "eeb891e3-3913-4590-82a9-dc23c212dceb", \n  "outcome": "True", \n  "predicted_outcome": "False"\n}\n'

<Response [200]>

...

b'{\n  "observation_id": "d46707aa-cf72-4fb4-aa64-fe6eff29e8f7", \n  "outcome": "True", \n  "predicted_outcome": "False"\n}\n'

<Response [200]>

b'{\n  "observation_id": "06e5e365-15a8-465a-8092-eef6762a78d7", \n  "outcome": "False", \n  "predicted_outcome": "False"\n}\n'

<Response [200]>

b'{\n  "observation_id": "89f0b741-dd82-4600-8b98-4c3230a0540d", \n  "outcome": "False", \n  "predicted_outcome": "True"\n}\n'

# Check data in the database

In [2]:
#server is down, so no worries (this credentials will fail, use yours)
username = 'postgres'
password = 'e4a0hDUQGnH26EDm82PK'
host_name = 'containers-us-west-118.railway.app'
port = 7357
db_name = 'railway'

conn_str = 'postgresql://{}:{}@{}:{}/{}'.format(username, password, host_name, port, db_name)
engine = sqlalchemy.create_engine(conn_str)
query = 'SELECT * FROM prediction;'
data_server = pd.DataFrame(engine.connect().execute(sqlalchemy.text(query)))
data_server

,id,observation_id,type,date,part_of_a_policing_operation,latitude,longitude,gender,age_range,officer_defined_ethnicity,legislation,object_of_search,station,proba,outcome,true_outcome
0,63,1b553034-0b2b-435b-b614-ce0a6c8d5f83,Person search,2022-01-01T01:05:06+00:00,False,52.574810,-0.241137,Male,10-17,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,cambridgeshire,0.467584,False,True
1,64,36ec1461-5190-496c-b82f-95c7ae9ff59f,Person search,2022-01-01T01:14:23+00:00,False,51.506256,-0.074901,Male,10-17,Asian,Police and Criminal Evidence Act 1984 (section 1),Offensive weapons,city-of-london,0.123517,False,False
2,65,6f38f480-c0a0-45fb-81cd-ab56f0bb6351,Person search,2022-01-01T01:20:32+00:00,False,51.506256,-0.074901,Male,10-17,Asian,Police and Criminal Evidence Act 1984 (section 1),Offensive weapons,city-of-london,0.123517,False,False
3,66,e7ab1073-5193-4ec6-aa07-efb0bc97acbb,Person search,2022-01-01T01:28:56+00:00,False,51.506256,-0.074901,Male,10-17,Asian,Police and Criminal Evidence Act 1984 (section 1),Offensive weapons,city-of-london,0.123517,False,False
4,67,ec925f54-c0dd-4527-b941-fcb87a12ec1c,Person search,2022-01-01T02:16:00+00:00,False,50.712350,-3.553934,Male,over 34,White,Police and Criminal Evidence Act 1984 (section 1),Offensive weapons,devon-and-cornwall,0.120817,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3902,2f15a5bb-ad8c-4986-8fce-383c33d1a4c3,Person search,2022-03-11T16:16:00+00:00,True,NaN,NaN,Male,10-17,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,nottinghamshire,0.851334,True,True
3996,3937,a2b1823d-dc5f-446a-9686-4c435f50d1a6,Person search,2022-03-11T23:11:00+00:00,False,NaN,NaN,Male,10-17,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,bedfordshire,0.546043,True,False
3997,3972,e2ff076c-5f29-4e95-8d97-75515d186480,Person and Vehicle search,2022-03-12T09:44:19+00:00,False,51.521810,-0.097929,Male,18-24,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,city-of-london,0.644289,True,True
3998,4006,f6563aaf-187f-4135-80c7-c11033cf657c,Person search,2022-03-13T12:14:31+00:00,False,51.513298,-0.101959,Male,18-24,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,city-of-london,0.559123,True,True


In [3]:
data_server.to_csv('data/requests1_final.csv')

In [6]:
data_server[["outcome","true_outcome"]].value_counts(dropna=False)

outcome  true_outcome
True     False           1941
False    False           1185
True     True             638
False    True             236
dtype: int64

In [4]:
data_server["true_outcome"].value_counts(dropna=False)

False    3126
True      874
Name: true_outcome, dtype: int64

In [22]:
df_

,observation_id,Type,Date,Part of a policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing,station
0,2e4d0094-c30b-471b-a211-72a9790feca2,Person search,2020-12-01T01:10:00+00:00,NaN,50.798824,-1.089471,Male,25-34,Other ethnic group - Not stated,White,Police and Criminal Evidence Act 1984 (section 1),Article for use in theft,Community resolution,False,False,hampshire
1,4779fbe8-6e05-4534-85fd-db32952ee309,Person search,2020-12-01T02:00:00+00:00,NaN,50.785099,-1.091540,Male,over 34,White - Any other White background,Other,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,False,False,hampshire
2,cb5c685d-acac-42e2-914d-75e6ff73b0a8,Person search,2020-12-01T09:15:00+00:00,NaN,50.952006,-1.403341,Male,over 34,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,False,True,hampshire
3,f486e116-5b1e-45db-9931-a7f070c5c478,Person search,2020-12-01T10:20:00+00:00,NaN,50.806383,-1.079844,Male,10-17,Other ethnic group - Not stated,White,Police and Criminal Evidence Act 1984 (section 1),Stolen goods,A no further action disposal,False,False,hampshire
4,78f4020e-12cc-4889-bf1a-2f2c29b2f662,Person search,2020-12-01T10:24:00+00:00,NaN,50.806670,-1.081982,Male,10-17,Other ethnic group - Not stated,Asian,Police and Criminal Evidence Act 1984 (section 1),Offensive weapons,A no further action disposal,False,False,hampshire
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856605,ee337b9a-12ad-45fd-8c60-49091a0f4ab8,Person and Vehicle search,2020-04-30T15:10:00+00:00,NaN,54.965502,-1.604609,Male,18-24,White - Any other White background,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,True,False,northumbria
856606,5973a004-e579-4dd2-bc26-71ab5717f87a,Person and Vehicle search,2020-04-30T15:10:00+00:00,NaN,54.965502,-1.604609,Male,25-34,White - Any other White background,Other,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,True,True,northumbria
856607,ad053a34-364e-4d24-8f5c-9734ab5fdbe0,Person and Vehicle search,2020-04-30T17:00:00+00:00,NaN,54.966266,-1.453704,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Khat or Cannabis warning,True,False,northumbria
856608,8736e5ec-7ca2-420b-ad56-4dd88d27fe6e,Person search,2020-04-30T17:35:00+00:00,NaN,54.971596,-1.636589,Male,25-34,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Arrest,True,False,northumbria


In [27]:
df_.loc[(df_['Outcome linked to object of search'] == False) & (df_['Object of search'] == 'Controlled drugs'), ['Outcome linked to object of search', 'Legislation', 'Outcome']].value_counts()

Outcome linked to object of search  Legislation                                              Outcome                        
False                               Misuse of Drugs Act 1971 (section 23)                    A no further action disposal       63508
                                                                                             Arrest                              7928
                                                                                             Community resolution                2852
                                                                                             Summons / charged by post           1623
                                                                                             Khat or Cannabis warning             668
                                                                                             Penalty Notice for Disorder          297
                                                                       

In [7]:
a = ['a' , 'b']
b = ['c', 'd']

z = [(x,y) for x in a for y in b]

z[0][1]

'c'